In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scanpy as sc

In [6]:
adata_asthma=sc.read('F:/Analysis/adata/adata_10asthma_unprocessed.h5ad')

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [7]:
adata_healthy=sc.read(r"D:/ISS/adata_baysor/adata_16samples_clustered_pearsonresidual_annotated_exc_TLN.h5ad")

AnnDataReadError: Above error raised while reading key '/layers' of type <class 'h5py._hl.group.Group'> from /.

In [36]:
adata

AnnData object with n_obs × n_vars = 38526 × 264
    obs: 'cell', 'x', 'y', 'cluster', 'n_transcripts', 'density', 'elongation', 'area', 'avg_confidence', 'sample', 'condition', 'number', 'sample_ID', 'total_counts', 'n_genes', 'leiden'
    obsm: 'X_pca', 'X_umap'

In [37]:
maximum_numbers_of_reads=100
minimum_number_of_reads=3
import seaborn as sns
adata.obs['total_counts']=list(np.sum(adata.X,axis=1))
#sns.displot(adata.obs,x='total_counts')
#plt.axvline(x=maximum_numbers_of_reads,color='green',linestyle='--')
#plt.axvline(x=minimum_number_of_reads,color='red',linestyle='--')
adata=adata[adata.obs['total_counts']>minimum_number_of_reads] #adjust the numbers to keep good cells according to appropiate ones
adata=adata[adata.obs['total_counts']<maximum_numbers_of_reads] #adjust the numbers to more than

In [38]:
adata

View of AnnData object with n_obs × n_vars = 36209 × 264
    obs: 'cell', 'x', 'y', 'cluster', 'n_transcripts', 'density', 'elongation', 'area', 'avg_confidence', 'sample', 'condition', 'number', 'sample_ID', 'total_counts', 'n_genes', 'leiden'
    obsm: 'X_pca', 'X_umap'

In [39]:
sc.pp.filter_cells(adata,min_genes=3)

Trying to set attribute `.obs` of view, copying.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [40]:
adata

AnnData object with n_obs × n_vars = 35758 × 264
    obs: 'cell', 'x', 'y', 'cluster', 'n_transcripts', 'density', 'elongation', 'area', 'avg_confidence', 'sample', 'condition', 'number', 'sample_ID', 'total_counts', 'n_genes', 'leiden'
    obsm: 'X_pca', 'X_umap'

In [41]:
adata.raw=adata
sc.pp.normalize_total(adata, target_sum=100)
sc.pp.log1p(adata)

In [42]:
sc.tl.pca(adata, svd_solver='arpack')
plt.rcParams['figure.facecolor'] = 'white'
sc.pl.pca_variance_ratio(adata, log=False)

In [43]:
#sc.pp.subsample(adata,0.1)

In [44]:
sc.pp.neighbors(adata, n_neighbors=10, n_pcs=0)

In [45]:
sc.tl.leiden(adata,resolution=1.0)

In [46]:
sc.tl.umap(adata,min_dist=0.01)

In [47]:
pd.crosstab(adata.obs['sample'],adata.obs['condition'])

condition,A,H
sample,,
REGION1,0,3110
REGION2,0,1524
REGION4,0,7762
REGION5,0,2408
REGION6,0,5412
REGION7,4598,0
REGION8,5156,0
REGION9,2236,0
REGION10,1704,0


In [ ]:
plt.rcParams['figure.facecolor'] = 'white'
sc.set_figure_params(figsize=(14,10),dpi=600)
sc.pl.umap(adata,color=['condition'],cmap='hsv',s=30)#,'total_counts'])#,save='chickbrain_UMAP_bins.pdf')

In [ ]:
plt.rcParams['figure.facecolor'] = 'white'
sc.set_figure_params(figsize=(14,10),dpi=600)
sc.pl.umap(adata,color=['sample'],cmap='hsv',s=30)#,'total_counts'])#,save='chickbrain_UMAP_bins.pdf')

In [ ]:
plt.rcParams['figure.facecolor'] = 'white'
sc.set_figure_params(figsize=(14,10),dpi=600)
sc.pl.umap(adata,color=['leiden'],cmap='hsv',s=30)#,'total_counts'])#,save='chickbrain_UMAP_bins.pdf')

In [62]:
cluster_by_sample=pd.crosstab(adata.obs['leiden'],adata.obs['sample'])

In [75]:
cdv=cluster_by_sample.div(cluster_by_sample.sum(axis=0),axis=1)

In [97]:
cd=cdv.div(cdv.sum(axis=1),axis=0)

In [ ]:
plt.figure(figsize=(7,7))
sns.heatmap(cd,vmax=0.5)

In [ ]:
adata.write('F:/Analysis/adata/adata_10asthma_clustered.h5ad')

In [ ]:
plt.rcParams['figure.facecolor'] = 'white'
sc.set_figure_params(figsize=(14,10),dpi=600)
sc.pl.umap(adata,color=['total_counts'],cmap='hsv',s=30)#,'total_counts'])#,save='chickbrain_UMAP_bins.pdf')

In [ ]:
sc.tl.rank_genes_groups(adata, 'leiden', method='wilcoxon')
sc.set_figure_params(figsize=(5,5),dpi=200)
sc.pl.rank_genes_groups(adata, n_genes=5, sharey=False,fontsize=14)

In [60]:
def map_of_clusters(adata,key='leiden',clusters='all',size=8,background='white',figuresize=(10,7),save=None,format='pdf'):
    
    #PARAMETERS TO MODIFY:
    #-key: the terms in adata.obs that you want to plot
    #-clusters:'all' for plotting all clusters in a single plot, 'individual': for plots of individual genes, or ['3','5'] (your groups
    #          between square brackets to plot only some clusters
    #-size: to change the size of your spots
    #-background: to change the color of the background
    #-figuresize: to specify the size of your figure
    #-save: if you want to save your figure, give the PATH of the folder where you want to save it
    #-format: specify the format in which you want to save your figure
    
    try:
        adata.obs[key]=adata.obs[key].astype(int)
        colors=dict(zip(np.unique(adata.obs[key]),adata.uns[key+'_colors']))
    except:
        colors=dict(zip(np.unique(adata.obs[key]),adata.uns[key+'_colors']))
    #cl.apply(lambda x: colors[x])
    plt.rcParams['figure.facecolor'] = background
    if clusters=='all':
        cl=adata.obs[key]
        plt.figure(figsize=figuresize)
        figa=plt.scatter(x=adata.obs.X,y=adata.obs.Y,c=cl.apply(lambda x: colors[x]),s=size,linewidths=0, edgecolors=None)
        plt.axis('off')
        if not save==None:
            plt.savefig(save +'/map_all_clusters_'+str(size)+'_'+background+'_'+key+'.'+format)
    elif clusters=='individual':
        cl=adata.obs[key]
        for each in adata.obs[key].unique():
            adatasub=adata[adata.obs[key]==each]
            plt.figure(figsize=figuresize)
            plt.scatter(x=adata.obs.X,y=adata.obs.Y,c='grey',s=size/5,linewidths=0, edgecolors=None)
            cl=adatasub.obs[key]
            plt.scatter(x=adatasub.obs.X,y=adatasub.obs.Y,c=cl.apply(lambda x: colors[x]),s=size,linewidths=0, edgecolors=None)
            plt.axis('off')
            plt.title('Group: '+ str(each))
            if not save==None:
                plt.savefig(save +'/map_inidivdual_cluster_'+str(each)+'_'+str(size)+background+'_'+key+'.'+format)
    else:
        adatasub=adata[adata.obs[key].isin(clusters)]
        plt.figure(figsize=figuresize)
        plt.scatter(x=adata.obs.X,y=adata.obs.Y,c='grey',s=size/5,linewidths=0, edgecolors=None)
        cl=adatasub.obs[key]
        plt.scatter(x=adatasub.obs.X,y=adatasub.obs.Y,c=cl.apply(lambda x: colors[x]),s=size,linewidths=0, edgecolors=None)
        plt.axis('off')
        plt.legend()
        if not save==None:
                s=''
                for element in clusters:
                    s=s+str(element)
                print(s)
                plt.savefig(save +'/map_group_of_clusters_'+str(s)+'_'+str(size)+background+'_'+key+'.'+format)
#        plt.title('Group: '+ paste(clusters))

In [ ]:
adata.obs['X']=adata.obs['x']
adata.obs['Y']=adata.obs['y']
for a in adata.obs['sample'].unique():
    ads=adata[adata.obs['sample']==a]
#    sc.pl.umap(ads,color=['leiden'],cmap='hsv',s=30)#,'total_counts'])#,save='chickbrain_UMAP_bins.pdf')
    map_of_clusters(ads,figuresize=(7,7),size=20,clusters='all')